In [ ]:
import sys
sys.path.append("../")

In [56]:
import pandas as pd
from datetime import datetime 
from pandas.tseries.offsets import CustomBusinessDay 
from pandas.tseries.holiday import USFederalHolidayCalendar

from CurvyCUSIPs.CurveDataFetcher import CurveDataFetcher
from CurvyCUSIPs.S490Swaps import S490Swaps
from CurvyCUSIPs.S490Swaptions import S490Swaptions 

import CurvyCUSIPs.HedgeHog.swaps as hh_swaps
import CurvyCUSIPs.HedgeHog.swaptions as hh_swaptions

import nest_asyncio
nest_asyncio.apply()

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
curve_data_fetcher = CurveDataFetcher()

In [38]:
s490_swaps = S490Swaps(s490_curve_db_path=r"..\db\nyclose_sofr_ois", curve_data_fetcher=curve_data_fetcher)
s490_swaptions = S490Swaptions(s490_swaps)

start_date = datetime(2024, 1, 1)
end_date = datetime(2025, 1, 2)
bdates = pd.date_range(start=start_date, end=end_date, freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))

In [39]:
fwd_tenors = ["1M", "3M", "6M", "9M", "12M", "18M", "2Y", "3Y", "4Y", "5Y", "6Y", "7Y", "8Y", "9Y", "10Y"]
fwd_grid_dict, ql_curves = s490_swaps.s490_nyclose_fwd_curve_matrices(
    start_date=start_date,
    end_date=end_date,
    ql_piecewise_method="logLinearDiscount",
    fwd_tenors=fwd_tenors,
)

Building Implied Fwd Curves...: 100%|██████████| 252/252 [00:02<00:00, 103.38it/s]


# Pricing 1mm 1Mx1Y Payer Swaption on 08/14/2024

![womp womp](../dump/8-14-2024-swpm.png)

## Grab EOD Discount Curve on Aug 14, 2024:

In [40]:
trade_date = datetime(2024, 8, 14)
fwd_grid_df = fwd_grid_dict[trade_date]

## Grab Vol Cube on Aug 14, 2024:

In [41]:
ql_vol_cube_handle = s490_swaptions.get_ql_vol_cube_handle(date=trade_date)

## Set-up pricer:

In [42]:
tenor = "1M Fwd 10Y"

book = [
    hh_swaps.SwapLeg(
        trade_date=trade_date,
        original_tenor=tenor,
        original_fixed_rate=fwd_grid_df[fwd_grid_df["Tenor"] == tenor.split(" Fwd ")[1]][
            f"{tenor.split(" Fwd ")[0]} Fwd"
        ].iloc[-1] / 100,
        weighting=1,
        key=tenor,
        notional=1_000_000,
        type="payer",
    )
]

In [43]:
swaption_book_metrics_dict = hh_swaptions.swaption_book_metrics(
    underlyings=book, ql_curve=ql_curves[trade_date], ql_yts=s490_swaps._ql_yts, ql_sofr=s490_swaps._ql_sofr, swaption_vol_handle=ql_vol_cube_handle 
)
swaption_book_metrics_dict

PRICING SWAPTIONS...: 100%|██████████| 3/3 [00:00<00:00,  3.56it/s]


{'1M Fwd 10Y': {'atm_strike': np.float64(0.033588757050794005),
  'strike': np.float64(0.033588757050794005),
  'strike_offset_bps': np.float64(0.0),
  'npv': np.float64(10632.227161208903),
  'normal_vol': np.float64(104.2977286881497),
  'bpvol': np.float64(6.57013934327931),
  'spot_prem_bps': 106.32227161208903,
  'fwd_prem_bps': 106.86420805913033,
  'ql': {'swaption': <QuantLib.QuantLib.Swaption; proxy of <Swig Object of type 'ext::shared_ptr< Swaption > *' at 0x00000275DE8D92F0> >,
   'underlying': <QuantLib.QuantLib.OvernightIndexedSwap; proxy of <Swig Object of type 'ext::shared_ptr< OvernightIndexedSwap > *' at 0x00000275DE8D8300> >},
  'greeks': {'dv01': np.float64(-423.7010371112129),
   'gamma_01': np.float64(10.655961322063147),
   'vega_01': np.float64(101.84320033808186),
   'volga_01': np.float64(0.3857203268609055),
   'vanna_01': np.float64(199.6035373667837),
   'theta_1d': np.float64(-161.06299231312914),
   'charm_1d': np.float64(-0.03266344307195368),
   'veta_1d

### I think that's a pretty good match given that we are using mostly public/open-sourced data!

# If you have a large book, use parallel pricer:
- SWIG objects can't be pickled for easy parallelization so theres a bit more overhead with unbundling the Quantlib object and them rebuilding them
- Usually gives a 2x speedup for larger books!

## Grab Vol Cube dictionary for Aug 14, 2024:

In [44]:
vol_cube_dict = s490_swaptions.get_vol_cube(date=trade_date)

## Set up parallel pricer:

In [45]:
tenor1 = "1M Fwd 10Y"
tenor2 = "3M Fwd 10Y"
tenor3 = "6M Fwd 10Y"
tenor4 = "12M Fwd 10Y"
tenor5 = "2Y Fwd 10Y"

book = [
    hh_swaps.SwapLeg(
        trade_date=trade_date,
        original_tenor=tenor1,
        original_fixed_rate=fwd_grid_df[fwd_grid_df["Tenor"] == tenor1.split(" Fwd ")[1]][
            f"{tenor1.split(" Fwd ")[0]} Fwd"
        ].iloc[-1] / 100,
        weighting=1,
        key=tenor1,
        notional=1_000_000,
        type="payer",
    ),
    hh_swaps.SwapLeg(
        trade_date=trade_date,
        original_tenor=tenor2,
        original_fixed_rate=fwd_grid_df[fwd_grid_df["Tenor"] == tenor2.split(" Fwd ")[1]][
            f"{tenor2.split(" Fwd ")[0]} Fwd"
        ].iloc[-1] / 100,
        weighting=1,
        key=tenor2,
        notional=100_000_000,
        type="receiver",
    ),
    hh_swaps.SwapLeg(
        trade_date=trade_date,
        original_tenor=tenor3,
        original_fixed_rate=fwd_grid_df[fwd_grid_df["Tenor"] == tenor3.split(" Fwd ")[1]][
            f"{tenor3.split(" Fwd ")[0]} Fwd"
        ].iloc[-1] / 100,
        weighting=1,
        key=tenor3,
        notional=10_000_000,
        type="payer",
    ),hh_swaps.SwapLeg(
        trade_date=trade_date,
        original_tenor=tenor4,
        original_fixed_rate=fwd_grid_df[fwd_grid_df["Tenor"] == tenor4.split(" Fwd ")[1]][
            f"{tenor4.split(" Fwd ")[0]} Fwd"
        ].iloc[-1] / 100,
        weighting=1,
        key=tenor4,
        notional=50_000_000,
        type="receiver",
    ),hh_swaps.SwapLeg(
        trade_date=trade_date,
        original_tenor=tenor5,
        original_fixed_rate=fwd_grid_df[fwd_grid_df["Tenor"] == tenor5.split(" Fwd ")[1]][
            f"{tenor5.split(" Fwd ")[0]} Fwd"
        ].iloc[-1] / 100,
        weighting=1,
        key=tenor5,
        notional=25_000_000,
        type="payer",
    )
]

In [49]:
swaption_book_metrics_dict = hh_swaptions.swaption_book_metrics_parallel(
    underlyings=book,
    ql_curve=ql_curves[trade_date],
    sofr_fixings_dates=s490_swaps._sofr_fixing_dates,
    sofr_fixings=s490_swaps._sofr_fixings,
    vol_cube_dict=vol_cube_dict,
    n_jobs=-1
)
swaption_book_metrics_dict

PRICING SWAPTIONS...: 100%|██████████| 5/5 [00:00<00:00, 5348.51it/s]
REBUILDING QL OBJECTS...: 5it [00:00, 118.53it/s]


{'1M Fwd 10Y': {'atm_strike': np.float64(0.033588757050794005),
  'strike': np.float64(0.033588757050794005),
  'strike_offset_bps': np.float64(0.0),
  'npv': np.float64(10632.227161208903),
  'normal_vol': np.float64(104.2977286881497),
  'bpvol': np.float64(6.57013934327931),
  'spot_prem_bps': 106.32227161208903,
  'fwd_prem_bps': 106.86420805913033,
  'greeks': {'dv01': np.float64(-423.7010371112129),
   'gamma_01': np.float64(10.655961322063147),
   'vega_01': np.float64(101.84320033808186),
   'volga_01': np.float64(0.3857203268609055),
   'vanna_01': np.float64(199.6035373667837),
   'theta_1d': np.float64(-161.06299231312914),
   'charm_1d': np.float64(-0.03266344307195368),
   'veta_1d': np.float64(-1.5550107946567948)},
  'ql': {'swaption': <QuantLib.QuantLib.Swaption; proxy of <Swig Object of type 'ext::shared_ptr< Swaption > *' at 0x00000275DE94D110> >,
   'underlying': <QuantLib.QuantLib.OvernightIndexedSwap; proxy of <Swig Object of type 'ext::shared_ptr< OvernightIndexed

In [48]:
swaption_book_metrics_dict = hh_swaptions.swaption_book_metrics(
    underlyings=book, ql_curve=ql_curves[trade_date], ql_yts=s490_swaps._ql_yts, ql_sofr=s490_swaps._ql_sofr, swaption_vol_handle=ql_vol_cube_handle 
)
swaption_book_metrics_dict

PRICING SWAPTIONS...: 100%|██████████| 15/15 [00:04<00:00,  3.53it/s]


{'1M Fwd 10Y': {'atm_strike': np.float64(0.033588757050794005),
  'strike': np.float64(0.033588757050794005),
  'strike_offset_bps': np.float64(0.0),
  'npv': np.float64(10632.227161208903),
  'normal_vol': np.float64(104.2977286881497),
  'bpvol': np.float64(6.57013934327931),
  'spot_prem_bps': 106.32227161208903,
  'fwd_prem_bps': 106.86420805913033,
  'ql': {'swaption': <QuantLib.QuantLib.Swaption; proxy of <Swig Object of type 'ext::shared_ptr< Swaption > *' at 0x00000275DE8D12F0> >,
   'underlying': <QuantLib.QuantLib.OvernightIndexedSwap; proxy of <Swig Object of type 'ext::shared_ptr< OvernightIndexedSwap > *' at 0x00000275DE8D3C00> >},
  'greeks': {'dv01': np.float64(-423.7010371112129),
   'gamma_01': np.float64(10.655961322063147),
   'vega_01': np.float64(101.84320033808186),
   'volga_01': np.float64(0.3857203268609055),
   'vanna_01': np.float64(199.6035373667837),
   'theta_1d': np.float64(-161.06299231312914),
   'charm_1d': np.float64(-0.03266344307195368),
   'veta_1d

## 3 second runtime for parallel pricer vs 6 second runtime

## Let's check another date:

![womp womp](../dump/2-23-2024-swpm.png)

## Grab curve and cube for 02/23/2024:

In [54]:
trade_date = datetime(2024, 2, 23)
fwd_grid_df = fwd_grid_dict[trade_date]
ql_vol_cube_handle = s490_swaptions.get_ql_vol_cube_handle(date=trade_date)

## Set up pricer:

In [57]:
tenor = "12M Fwd 5Y"

book = [
    hh_swaps.SwapLeg(
        trade_date=trade_date,
        original_tenor=tenor,
        original_fixed_rate=fwd_grid_df[fwd_grid_df["Tenor"] == tenor.split(" Fwd ")[1]][
            f"{tenor.split(" Fwd ")[0]} Fwd"
        ].iloc[-1] / 100,
        weighting=1,
        key=tenor,
        notional=100_000_000,
        type="payer",
    )
]

swaption_book_metrics_dict = hh_swaptions.swaption_book_metrics(
    underlyings=book, ql_curve=ql_curves[trade_date], ql_yts=s490_swaps._ql_yts, ql_sofr=s490_swaps._ql_sofr, swaption_vol_handle=ql_vol_cube_handle 
)
swaption_book_metrics_dict

PRICING SWAPTIONS...: 100%|██████████| 3/3 [00:00<00:00, 12.44it/s]


{'12M Fwd 5Y': {'atm_strike': np.float64(0.037013695773346456),
  'strike': np.float64(0.037013695773346456),
  'strike_offset_bps': np.float64(0.0),
  'npv': np.float64(2060738.2201328082),
  'normal_vol': np.float64(118.81406042852984),
  'bpvol': np.float64(7.484582289325959),
  'spot_prem_bps': 206.07382201328082,
  'fwd_prem_bps': 216.772668209519,
  'ql': {'swaption': <QuantLib.QuantLib.Swaption; proxy of <Swig Object of type 'ext::shared_ptr< Swaption > *' at 0x00000275DE92D0B0> >,
   'underlying': <QuantLib.QuantLib.OvernightIndexedSwap; proxy of <Swig Object of type 'ext::shared_ptr< OvernightIndexedSwap > *' at 0x00000275DE92EB80> >},
  'greeks': {'dv01': np.float64(-21637.987403530213),
   'gamma_01': np.float64(144.8517770871822),
   'vega_01': np.float64(17374.65405790104),
   'volga_01': np.float64(24.772794622549554),
   'vanna_01': np.float64(6038.289224961773),
   'theta_1d': np.float64(-8062.404142011888),
   'charm_1d': np.float64(-0.27864800465249573),
   'veta_1d':

## Note: OIS DC Stripping was not used in the SWPM screen grab so `npv` and `theta_1d` are expectedly different
- recall that early 2024, market was pricing in massive rate cuts (at some point 6+ 25bp cuts)
- since SWPM isn't using dual curve ois discounting, it's not accounting for the downshifted forward curve
- since we are pricing a payer (call on rates), SWPM without OIS DC STRIPPING is over valuing the swaption
- i.e. with deep rate cuts, the forward curve and volatility surface align in a way that accelerates the reduction in the swaption's value over time => higher time decay
- since our pricer uses OIS DC Stripping, our NPV and notably theta is much lower